In [1]:
import numpy as np
import torch

from s3ts.helper_functions import *

In [8]:
parser = get_parser()
args = parser.parse_args([])

In [20]:
args.dataset = "WISDM"
args.dataset_dir = "./datasets/"
args.rho = 0.1
args.batch_size = 128
args.num_workers = 8
args.num_medoids = 2
args.window_size = 48
args.window_stride = 1
args.normalize = True
args.pattern_size = 24
args.compute_n = 250
args.subjects_for_test = [30, 31, 32, 33, 34, 35]
args.mode = "img"
args.reduce_imbalance = True
args.label_mode = 1
args.voting = 1
args.overlap = 45
args.encoder_architecture = "cnn_gap"
args.encoder_features = 20
args.decoder_architecture = "mlp"
args.decoder_features = 64
args.decoder_layers = 1

In [21]:
dm = load_dm(args)

Loaded dataset WISDM with a total of 1096480 observations for window size 48
Computing medoids...
Computing dissimilarity frames...
Using 148133 observations for training and 51459 observations for validation and test


In [22]:
from s3ts.api.nets.methods import train_model

model = get_model("test", args, dm)

Input shape:  torch.Size([1, 12, 24, 48])
Latent shape:  torch.Size([1, 80])
[tensor(80), 64, 6]


In [23]:
train_model(dm, model, 1, {
    "default_root_dir": "training",
    "accelerator": "auto",
    "seed": 42
})

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type                      | Params
---------------------------------------------------------
0 | encoder    | CNN_GAP_IMG               | 23.1 K
1 | decoder    | MultiLayerPerceptron      | 5.6 K 
2 | flatten    | Flatten                   | 0     
3 | softmax    | Softmax                   | 0     
4 | train_cm   | MulticlassConfusionMatrix | 0     
5 | val_cm     | MulticlassConfusionMatrix | 0     
6 | val_auroc  | MulticlassAUROC           | 0     
7 | test_cm    | MulticlassConfusionMatrix | 0     
8 | test_auroc | MulticlassAUROC           | 0     
---------------------------------------------------------
28.7 K    Trainable params
0         Non-trainable params
28.7 K    Total params
0.115     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 1158/1158 [02:32<00:00,  7.60it/s, v_num=25, train_loss_step=0.464, val_loss=0.938, val_auroc=0.890, val_re=0.601, train_loss_epoch=0.542]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 1158/1158 [02:32<00:00,  7.60it/s, v_num=25, train_loss_step=0.464, val_loss=0.938, val_auroc=0.890, val_re=0.601, train_loss_epoch=0.542]
Input shape:  torch.Size([1, 12, 24, 48])
Latent shape:  torch.Size([1, 80])
[tensor(80), 64, 6]
Validation DataLoader 0: 100%|██████████| 403/403 [00:24<00:00, 16.60it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_auroc         │    0.8895742297172546     │
│          val_f1           │    0.5791109204292297     │
│         val_loss          │    0.9381545782089233     │
│          val_pr           │    0.5894643068313599     │
│          val_re           │    0.6007785797119141     │
└───────────────────────────┴───────────────────────────┘

(WrapperModel(
   (encoder): CNN_GAP_IMG(
     (cnn_0): Sequential(
       (0): Conv2d(12, 20, kernel_size=(5, 5), stride=(1, 1), padding=same)
       (1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
       (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     )
     (cnn_1): Sequential(
       (0): Conv2d(20, 20, kernel_size=(4, 4), stride=(1, 1), padding=same)
       (1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
       (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     )
     (cnn_2): Sequential(
       (0): Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1), padding=valid)
       (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
     )
     (last): Sequential(
       (0): Conv2d(40, 80, kernel_size=(1, 1), stride=(1, 1), padding=same)
       (1): ReLU()
   